In [41]:
from keras.applications import VGG19
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
from sklearn.model_selection import train_test_split

In [31]:
data_te = ImageDataGenerator(rescale = 1./255).flow_from_directory('DATASET_2/TEST', 
                                                                   target_size = (224, 224), 
                                                                   batch_size = 2513,
                                                                   seed = 123)

data_tr = ImageDataGenerator(rescale = 1./255).flow_from_directory('DATASET_2/TRAIN', 
                                                                   target_size = (224, 224),  
                                                                   batch_size = 22564,
                                                                   seed = 123)

Found 2513 images belonging to 2 classes.
Found 22564 images belonging to 2 classes.


In [32]:
images_te, labels_te = next(data_te)
images_tr, labels_tr = next(data_tr)

In [33]:
print(images_te.shape)
print(images_tr.shape)

(2513, 224, 224, 3)
(22564, 224, 224, 3)


In [35]:
print(labels_te.shape)
print(labels_tr.shape)

(2513, 2)
(22564, 2)


In [36]:
images = np.concatenate((images_tr, images_te))
images.shape

(25077, 224, 224, 3)

In [39]:
labels = np.concatenate((labels_tr[:,0], labels_te[:,0]))
labels.shape

(25077,)

In [40]:
X_model, X_test, y_model, y_test = train_test_split(images, labels, test_size = 0.3, random_state = 123)
X_train, X_val, y_train, y_val = train_test_split(X_model, y_model, test_size = 0.3, random_state = 123)